<a href="https://colab.research.google.com/github/krish-navulla/FL_Breast_Cancer_Classification/blob/main/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import pandas as pd
from PIL import Image
import os
import pandas as pd
import torch
from torchvision.io import read_image
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from tqdm import tqdm



# Creating pytorch data loader

# Creating annotations file and client datasets

In [17]:
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision import transforms

annotations_benign = pd.read_csv("/content/drive/MyDrive/Thesis/BreaKHis_v1/histology_slides/breast/annotations_benign.txt", delimiter='\t')
annotations_malignant = pd.read_csv("/content/drive/MyDrive/Thesis/BreaKHis_v1/histology_slides/breast/annotations_malignant.txt", delimiter='\t')

annotations = pd.concat([pd.DataFrame(annotations_malignant, columns=['name', 'value']), pd.DataFrame(annotations_benign, columns=['name', 'value'])])

annotations = annotations.sample(frac=1).reset_index(drop=True)

class CustomImageDataset():
    def __init__(self, annotations_file, img_dir, transform, split_ratios, target_transform=None):
        self.img_labels = annotations_file.reset_index(drop=True)
        print(f"Length of DataFrame: {len(self.img_labels)}")  # Print the length of the DataFrame
        print(f"Sample of DataFrame:\n{self.img_labels.head()}")  # Print a sample of the DataFrame
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.split_ratios = split_ratios

        # Split the dataset into subsets for each client
        self._split_dataset()

    def _split_dataset(self):
        if self.split_ratios is None:
            raise ValueError("split_ratios cannot be None")

        # Calculate the sizes of the splits for each client
        split_sizes = [int(len(self.img_labels) * ratio) for ratio in self.split_ratios]

        # Split the dataset into subsets for each client
        self.datasets = []
        start_idx = 0
        for size in split_sizes:
            subset = self.img_labels.iloc[start_idx:start_idx + size]
            self.datasets.append(subset)
            start_idx += size

    def __len__(self):
        return len(self.img_labels)

    # def __getitem__(self, idx):
    #     if idx >= len(self.img_labels):
    #         raise IndexError(f"Index {idx} is out of range for the dataset")
    #     img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0]).replace("\\","/")
    #     image = Image.open(img_path).convert('RGB')
    #     label = self.img_labels.iloc[idx, 1]
    #     if self.transform:
    #         image = self.transform(image)
    #     if self.target_transform:
    #         label = self.target_transform(label)
    #     return image, label
    def __getitem__(self, idx):
        try:
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0]).replace("\\","/")
            image = Image.open(img_path).convert('RGB')
            label = self.img_labels.iloc[idx, 1]
            if self.transform:
                image = self.transform(image)
            if self.target_transform:
                label = self.target_transform(label)
            return image, label
        except KeyError as e:
            print(f"KeyError: {e}")
            print(f"Index: {idx}, Length of DataFrame: {len(self.img_labels)}")
            raise e
        except IndexError as e:
            print(f"IndexError: {e}")
            print(f"Index: {idx}, Length of DataFrame: {len(self.img_labels)}")
            raise e


# Define split ratios for each client
split_ratios = [0.25, 0.25, 0.25, 0.25]  # Adjust the split ratios as needed

img_dir = "\content\drive\MyDrive\Thesis\BreaKHis_v1"
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust the size as needed
    transforms.ToTensor(),
])

dataset = CustomImageDataset(
    img_dir=img_dir,
    annotations_file=annotations,
    transform=transform,
    split_ratios=split_ratios
)

# Access datasets for each client
client_datasets =dataset.datasets


SyntaxError: non-default argument follows default argument (<ipython-input-17-d018823f0639>, line 8)

In [ ]:
# client_datasets = {'client_1': client_datasets[0].reset_index(drop=True), 'client_2': client_datasets[1].reset_index(drop=True), 'client_3': client_datasets[2].reset_index(drop=True), 'client_4': client_datasets[3].reset_index(drop=True)}
client_datasets = {'client_1': client_datasets[0], 'client_2': client_datasets[1], 'client_3': client_datasets[2], 'client_4': client_datasets[3]}

# Test model


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from tqdm import tqdm

# Client 1

In [ ]:
train_size = int(0.7 * len(client1_dataset))
val_size = int(0.15 * len(client1_dataset))
test_size = len(client1_dataset) - train_size - val_size
num_epochs = 2

train_dataset, val_dataset, test_dataset = random_split(client1_dataset, [train_size, val_size, test_size])

# Create DataLoader instances for each set
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
client_1_model = models.resnet18(pretrained=True)
num_ftrs = client_1_model.fc.in_features
client_1_model.fc = nn.Linear(num_ftrs, 2)  # Assuming you have 2 classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
client_1_model = client_1_model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(client_1_model.parameters(), lr=0.001)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    client_1_model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = client_1_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Train Loss: {epoch_loss}")

    # Evaluation on validation set
    client_1_model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = client_1_model(images)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    val_accuracy = 100 * val_correct / val_total
    print(f"Validation Accuracy: {val_accuracy:.2f}%")

# Test the model on the test set
client_1_model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = client_1_model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy: {test_accuracy:.2f}%")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/2: 100%|██████████| 22/22 [19:50<00:00, 54.10s/it]


Train Loss: 0.5510490213023177
Validation Accuracy: 71.62%


Epoch 2/2: 100%|██████████| 22/22 [00:25<00:00,  1.16s/it]


Train Loss: 0.30845355926134066
Validation Accuracy: 85.47%
Test Accuracy: 87.58%


# Client 2

In [ ]:
train_size = int(0.7 * len(client2_dataset))
val_size = int(0.15 * len(client2_dataset))
test_size = len(client2_dataset) - train_size - val_size
num_epochs = 2

train_dataset, val_dataset, test_dataset = random_split(client2_dataset, [train_size, val_size, test_size])

# Create DataLoader instances for each set
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
client_2_model = models.resnet18(pretrained=True)
num_ftrs = client_2_model.fc.in_features
client_2_model.fc = nn.Linear(num_ftrs, 2)  # Assuming you have 2 classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
client_2_model = client_2_model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(client_2_model.parameters(), lr=0.001)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    client_2_model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = client_2_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Train Loss client_2_model: {epoch_loss}")

    # Evaluation on validation set
    client_2_model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = client_2_model(images)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    val_accuracy = 100 * val_correct / val_total
    print(f"Validation Accuracy client_2_model: {val_accuracy:.2f}%")

# Test the model on the test set
client_2_model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = client_2_model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy client_2_model: {test_accuracy:.2f}%")

Epoch 1/2: 100%|██████████| 22/22 [19:54<00:00, 54.30s/it]


Train Loss client_2_model: 0.5302637593951333
Validation Accuracy client_2_model: 68.92%


Epoch 2/2: 100%|██████████| 22/22 [00:25<00:00,  1.17s/it]


Train Loss client_2_model: 0.2728493737295238
Validation Accuracy client_2_model: 88.51%
Test Accuracy client_2_model: 87.58%


# Client 3

In [ ]:
train_size = int(0.7 * len(client1_dataset))
val_size = int(0.15 * len(client1_dataset))
test_size = len(client1_dataset) - train_size - val_size
num_epochs = 2

train_dataset, val_dataset, test_dataset = random_split(client1_dataset, [train_size, val_size, test_size])

# Create DataLoader instances for each set
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
client_3_model = models.resnet18(pretrained=True)
num_ftrs = client_3_model.fc.in_features
client_3_model.fc = nn.Linear(num_ftrs, 2)  # Assuming you have 2 classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
client_3_model = client_3_model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(client_3_model.parameters(), lr=0.001)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    client_3_model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = client_3_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Train Loss client_3_model: {epoch_loss}")

    # Evaluation on validation set
    client_3_model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = client_3_model(images)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    val_accuracy = 100 * val_correct / val_total
    print(f"Validation Accuracy client_3_model: {val_accuracy:.2f}%")

# Test the model on the test set
client_3_model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = client_3_model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy client_3_model: {test_accuracy:.2f}%")

Epoch 1/2: 100%|██████████| 22/22 [00:25<00:00,  1.17s/it]


Train Loss client_3_model: 0.47678458569968685
Validation Accuracy client_3_model: 76.69%


Epoch 2/2: 100%|██████████| 22/22 [00:25<00:00,  1.16s/it]


Train Loss client_3_model: 0.243444893425647
Validation Accuracy client_3_model: 83.45%
Test Accuracy client_3_model: 87.25%


# Client 4

In [ ]:
train_size = int(0.7 * len(client1_dataset))
val_size = int(0.15 * len(client1_dataset))
test_size = len(client1_dataset) - train_size - val_size
num_epochs = 2

train_dataset, val_dataset, test_dataset = random_split(client1_dataset, [train_size, val_size, test_size])

# Create DataLoader instances for each set
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
client_4_model = models.resnet18(pretrained=True)
num_ftrs = client_4_model.fc.in_features
client_4_model.fc = nn.Linear(num_ftrs, 2)  # Assuming you have 2 classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
client_4_model = client_4_model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(client_4_model.parameters(), lr=0.001)

# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    client_4_model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = client_4_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Train Loss client_4_model: {epoch_loss}")

    # Evaluation on validation set
    client_4_model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = client_4_model(images)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    val_accuracy = 100 * val_correct / val_total
    print(f"Validation Accuracy client_4_model: {val_accuracy:.2f}%")

# Test the model on the test set
client_4_model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = client_4_model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy client_4_model: {test_accuracy:.2f}%")

Epoch 1/2: 100%|██████████| 22/22 [00:25<00:00,  1.16s/it]


Train Loss client_4_model: 0.5318356481739687
Validation Accuracy client_4_model: 75.68%


Epoch 2/2: 100%|██████████| 22/22 [00:25<00:00,  1.16s/it]


Train Loss client_4_model: 0.27034093614126237
Validation Accuracy client_4_model: 88.18%
Test Accuracy client_4_model: 86.24%


# Saving Model Weights

In [ ]:
# Get the state_dict of the trained model
client_1_model_weights = client_1_model.state_dict()
client_2_model_weights = client_2_model.state_dict()
client_3_model_weights = client_3_model.state_dict()
client_4_model_weights = client_4_model.state_dict()

# Save the model weights to a file
torch.save(client_1_model_weights, 'client_1_model_weights.pth')
torch.save(client_2_model_weights, 'client_2_model_weights.pth')
torch.save(client_3_model_weights, 'client_3_model_weights.pth')
torch.save(client_4_model_weights, 'client_4_model_weights.pth')



# Weighted Average

In [ ]:
weight_client_1 = 0.25
weight_client_2 = 0.25
weight_client_3 = 0.25
weight_client_4 = 0.25

# Perform weighted average of the model weights
weighted_average_weights = {}
for key in client_1_model_weights.keys():
    weighted_average_weights[key] = (
        weight_client_1 * client_1_model_weights[key] +
        weight_client_2 * client_2_model_weights[key] +
        weight_client_3 * client_3_model_weights[key] +
        weight_client_4 * client_4_model_weights[key]
    )

# Save the weighted average model weights to a file
torch.save(weighted_average_weights, 'weighted_average_model_weights.pth')

# Weighted Averaged model

In [ ]:
import copy
weighted_model = copy.deepcopy(client_4_model)


In [ ]:





device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weighted_model = weighted_model.to(device)

weighted_average_model_weights = torch.load('/content/weighted_average_model_weights.pth')
weighted_model.load_state_dict(weighted_average_model_weights)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(weighted_model.parameters(), lr=0.001)

# Test the model on the test set
weighted_model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = weighted_model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy weighted_model: {test_accuracy:.2f}%")

Test Accuracy weighted_model: 67.11%


# Normalized weighted scheme

In [ ]:
# Validation accuracies of each client model
validation_accuracies = {
    'client_1_model': 85.47,
    'client_2_model': 88.51,
    'client_3_model': 83.45,
    'client_4_model': 88.18
}

# Normalize validation accuracies
total_accuracy = sum(validation_accuracies.values())
normalized_accuracies = {client: accuracy / total_accuracy for client, accuracy in validation_accuracies.items()}

# Assign weights based on normalized accuracies
weights = {client: normalized_accuracy for client, normalized_accuracy in normalized_accuracies.items()}

print("Weighting Scheme:")
for client, weight in weights.items():
    print(f"{client}: {weight:.2f}")


Weighting Scheme:
client_1_model: 0.25
client_2_model: 0.26
client_3_model: 0.24
client_4_model: 0.26


In [ ]:
weight_client_1_normalized_validation = 0.25
weight_client_2_normalized_validation = 0.26
weight_client_3_normalized_validation = 0.24
weight_client_4_normalized_validation = 0.26

weighted_average_weights = {}
for key in client_1_model_weights.keys():
    weighted_average_weights[key] = (
        weight_client_1_normalized_validation * client_1_model_weights[key] +
        weight_client_2_normalized_validation * client_2_model_weights[key] +
        weight_client_3_normalized_validation * client_3_model_weights[key] +
        weight_client_4_normalized_validation * client_4_model_weights[key]
    )

# Save the weighted average model weights to a file
torch.save(weighted_average_weights, 'Normalized_validation_weighted_average_model_weights.pth')

# Weighted Model - Normalized Validation scheme for weights

In [ ]:
weighted_model_NV = copy.deepcopy(client_4_model)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weighted_model_NV = weighted_model_NV.to(device)

weighted_average_model_weights = torch.load('/content/Normalized_validation_weighted_average_model_weights.pth')
weighted_model_NV.load_state_dict(weighted_average_model_weights)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(weighted_model_NV.parameters(), lr=0.001)

# Test the model on the test set
weighted_model_NV.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = weighted_model_NV(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
test_accuracy = 100 * test_correct / test_total
print(f"Test Accuracy weighted_model_NV: {test_accuracy:.2f}%")

Test Accuracy weighted_model_NV: 67.11%


In [ ]:
import shutil

In [ ]:

source_path = '/content/Normalized_validation_weighted_average_model_weights.pth'
destination_path = '/content/drive/MyDrive/Thesis/'
shutil.move(source_path, destination_path)


'/content/drive/MyDrive/Thesis/Normalized_validation_weighted_average_model_weights.pth'

# FL Cyles = 4

### Method definations

In [7]:
def train_model(model, dataset, batch_size,model_name, num_epochs=2, device="cpu", test_only = False):

    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size

    # Split the dataset
    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

    # Create DataLoader instances for each set
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Define the optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)  # Assuming you have 2 classes

    if test_only:
      model.to(device)
      model.eval()
      test_correct = 0
      test_total = 0
      with torch.no_grad():
          for images, labels in test_loader:
              images, labels = images.to(device), labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs, 1)
              test_total += labels.size(0)
              test_correct += (predicted == labels).sum().item()
      test_accuracy = 100 * test_correct / test_total
      # print(f"Test Accuracy {model_name} : {test_accuracy:.2f}%")
      return test_accuracy

    else:
    # Training loop
      model = model.to(device)
      for epoch in range(num_epochs):
          model.train()
          running_loss = 0.0
          for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
              images, labels = images.to(device), labels.to(device)
              optimizer.zero_grad()
              outputs = model(images)
              loss = criterion(outputs, labels)
              loss.backward()
              optimizer.step()
              running_loss += loss.item() * images.size(0)
          epoch_loss = running_loss / len(train_loader.dataset)
          print(f"Train Loss {model_name} : {epoch_loss}")

          # Evaluation on validation set
          model.eval()
          val_correct = 0
          val_total = 0
          with torch.no_grad():
              for images, labels in val_loader:
                  images, labels = images.to(device), labels.to(device)
                  outputs = model(images)
                  _, predicted = torch.max(outputs, 1)
                  val_total += labels.size(0)
                  val_correct += (predicted == labels).sum().item()
          val_accuracy = 100 * val_correct / val_total
          print(f"Validation Accuracy {model_name} : {val_accuracy:.2f}%")

      # Test the model on the test set
      model.eval()
      test_correct = 0
      test_total = 0
      with torch.no_grad():
          for images, labels in test_loader:
              images, labels = images.to(device), labels.to(device)
              outputs = model(images)
              _, predicted = torch.max(outputs, 1)
              test_total += labels.size(0)
              test_correct += (predicted == labels).sum().item()
      test_accuracy = 100 * test_correct / test_total
      print(f"Test Accuracy {model_name} : {test_accuracy:.2f}%")

      torch.save(model.state_dict(), f"{model_name}_model_weights.pth")



def compute_weighted_average_model(model_names, weights, cycle_number):
    # Load the saved model weights
    client_model_weights = {}
    for model_name in model_names:
        client_model_weights[model_name] = torch.load(f'{model_name}_model_weights.pth')

    # Perform weighted average of the model weights
    weighted_average_weights = {}
    for key in client_model_weights[model_names[0]].keys():
        weighted_average_weights[key] = sum(weights[model_name] * client_model_weights[model_name][key] for model_name in model_names)

    # Save the weighted average weights to a file
    torch.save(weighted_average_weights, f'weighted_average_model_weights_cycle_{cycle_number}.pth')



model_names = ['client_1', 'client_2', 'client_3', 'client_4']
weights = {'client_1': 0.25, 'client_2': 0.25, 'client_3': 0.25, 'client_4': 0.25}


def federated_learning(num_cycles,model_names, weights, num_epochs=2, batch_size=64):

  for i in range(num_cycles):
    if i == 0:
      weighted_average_weights = torch.load("/content/drive/MyDrive/Thesis/weighted_average_model_weights.pth")
    else:
      weighted_average_weights = torch.load(f'weighted_average_model_weights_cycle_{i}.pth')
    print(f"Federated Learning Cycle {i+1}")
    for model_name in model_names:
      print(f"Training {model_name} model")
      model = models.resnet18(pretrained=True)
      num_ftrs = model.fc.in_features
      model.fc = nn.Linear(num_ftrs, 2)  # Assuming you have 2 classes
      model.load_state_dict(weighted_average_weights)
      train_model(model, client_datasets[model_name], batch_size, model_name = model_name, num_epochs=0, device="cpu",test_only=False )

    compute_weighted_average_model(model_names, weights, cycle_number = i)
    print("Weighted average model weights computed")

    print(f"Testing FL model")
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)  # Assuming you have 2 classes
    model.load_state_dict(f'weighted_average_model_weights_cycle_{i}.pth')
    train_model(model, client_datasets['client_1'], batch_size, model_name = model_name, num_epochs=2, device="cpu",test_only=True )


federated_learning(2, model_names, weights, num_epochs=2, batch_size=64)



Federated Learning Cycle 1
Training client_1 model


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 27.3MB/s]


KeyError: 1413

# 1

In [16]:
import logging
from PIL import Image

# Set up logging
logging.basicConfig(level=logging.DEBUG)

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations_file.reset_index(drop=True)
        logging.debug(f"Initial dataset length: {len(self.img_labels)}")
        logging.debug(f"Sample of DataFrame:\n{self.img_labels.head()}")
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        try:
            if idx >= len(self.img_labels):
                raise IndexError(f"Index {idx} is out of range for the dataset")

            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0]).replace("\\", "/")
            image = Image.open(img_path).convert('RGB')
            label = self.img_labels.iloc[idx, 1]

            logging.debug(f"Loading image at {img_path}, label: {label}")

            if self.transform:
                image = self.transform(image)
            if self.target_transform:
                label = self.target_transform(label)

            return image, label

        except Exception as e:
            logging.error(f"Error loading data at index {idx}: {e}")
            raise e

# Function to split dataset into clients
def split_dataset(annotations, split_ratios):
    total_length = len(annotations)
    split_indices = [int(ratio * total_length) for ratio in split_ratios]

    datasets = []
    start_idx = 0
    for size in split_indices:
        datasets.append(annotations.iloc[start_idx:start_idx + size].reset_index(drop=True))
        start_idx += size

    return datasets

def train_model(model, dataset, batch_size, model_name, num_epochs, device="cuda", test_only=False):
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size

    train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

    logging.debug(f"Train size: {train_size}, Val size: {val_size}, Test size: {test_size}")

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)

    if test_only:
        model.to(device)
        model.eval()
        test_correct = 0
        test_total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
        test_accuracy = 100 * test_correct / test_total

        return test_accuracy
    else:
        model = model.to(device)
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * images.size(0)
            epoch_loss = running_loss / len(train_loader.dataset)
            print(f"Train Loss {model_name}: {epoch_loss}")

            model.eval()
            val_correct = 0
            val_total = 0
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    _, predicted = torch.max(outputs, 1)
                    val_total += labels.size(0)
                    val_correct += (predicted == labels).sum().item()
            val_accuracy = 100 * val_correct / val_total
            print(f"Validation Accuracy {model_name}: {val_accuracy:.2f}%")

        model.eval()
        test_correct = 0
        test_total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
        test_accuracy = 100 * test_correct / test_total
        print(f"Test Accuracy {model_name}: {test_accuracy:.2f}%")

        torch.save(model.state_dict(), f"{model_name}_model_weights.pth")


def compute_weighted_average_model(model_names, weights, cycle_number):
    # Load the saved model weights
    client_model_weights = {}
    for model_name in model_names:
        client_model_weights[model_name] = torch.load(f'{model_name}_model_weights.pth')

    # Perform weighted average of the model weights
    weighted_average_weights = {}
    for key in client_model_weights[model_names[0]].keys():
        weighted_average_weights[key] = sum(weights[model_name] * client_model_weights[model_name][key] for model_name in model_names)

    # Save the weighted average weights to a file
    torch.save(weighted_average_weights, f'weighted_average_model_weights_cycle_{cycle_number}.pth')


annotations_benign = pd.read_csv("/content/drive/MyDrive/Thesis/BreaKHis_v1/histology_slides/breast/annotations_benign.txt", delimiter='\t')
annotations_malignant = pd.read_csv("/content/drive/MyDrive/Thesis/BreaKHis_v1/histology_slides/breast/annotations_malignant.txt", delimiter='\t')

annotations = pd.concat([pd.DataFrame(annotations_malignant, columns=['name', 'value']), pd.DataFrame(annotations_benign, columns=['name', 'value'])])

annotations = annotations.sample(frac=1).reset_index(drop=True)

split_ratios = [0.25, 0.25, 0.25, 0.25]
img_dir = "/content/drive/MyDrive/Thesis/BreaKHis_v1"
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Split dataset into client datasets
client_datasets_dataframes = split_dataset(annotations, split_ratios)
client_datasets = {}
for i, df in enumerate(client_datasets_dataframes):
    client_datasets[f'client_{i+1}'] = CustomImageDataset(df, img_dir, transform=transform)




model_names = ['client_1', 'client_2', 'client_3', 'client_4']
weights = {'client_1': 0.25, 'client_2': 0.25, 'client_3': 0.25, 'client_4': 0.25}


def federated_learning(num_cycles,model_names, weights, num_epochs=2, batch_size=64):

  for i in range(num_cycles):
    if i == 0:
      weighted_average_weights = torch.load("/content/drive/MyDrive/Thesis/weighted_average_model_weights.pth")
    else:
      weighted_average_weights = torch.load(f'weighted_average_model_weights_cycle_{i-1}.pth')
    print(f"Federated Learning Cycle {i+1}")
    for model_name in model_names:
      print(f"Training {model_name} model")
      model = models.resnet18(pretrained=True)
      num_ftrs = model.fc.in_features
      model.fc = nn.Linear(num_ftrs, 2)  # Assuming you have 2 classes
      model.load_state_dict(weighted_average_weights)
      train_model(model, client_datasets[model_name], batch_size, model_name = model_name, num_epochs=2, device="cuda",test_only=False )

    compute_weighted_average_model(model_names, weights, cycle_number = i)
    print("Weighted average model weights computed")

    print(f"Testing FL model")
    model = models.resnet18(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)  # Assuming you have 2 classes
    model.load_state_dict(torch.load(f'weighted_average_model_weights_cycle_{i}.pth'))
    accuracy = train_model(model, client_datasets['client_1'], batch_size, model_name = model_names[0], num_epochs=0, device="cuda",test_only=True )
    print(f"Test Accuracy of {model_names[0]} with Federated weights after cycle {i}: {accuracy:.2f}%")

federated_learning(2, model_names, weights, num_epochs=2, batch_size=64)

Federated Learning Cycle 1
Training client_1 model


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/2: 100%|██████████| 22/22 [14:06<00:00, 38.49s/it]


Train Loss client_1: 0.38982379804981504
Validation Accuracy client_1: 76.69%


Epoch 2/2: 100%|██████████| 22/22 [00:26<00:00,  1.19s/it]


Train Loss client_1: 0.2651934500673243
Validation Accuracy client_1: 83.78%
Test Accuracy client_1: 84.23%
Training client_2 model


...

Epoch 1/2: 100%|██████████| 22/22 [14:44<00:00, 40.22s/it]


Train Loss client_4: 0.459792381361267
Validation Accuracy client_4: 88.85%


Epoch 2/2: 100%|██████████| 22/22 [00:25<00:00,  1.16s/it]


Train Loss client_4: 0.23724858177285735
Validation Accuracy client_4: 80.07%
Test Accuracy client_4: 83.22%
Weighted average model weights computed
Testing FL model
Test Accuracy of c with Federated weights after cycle 0: 33.56%


FileNotFoundError: [Errno 2] No such file or directory: 'weighted_average_model_weights_cycle_1.pth'

In [6]:
for i in range(5):
  print(f"Test Accuracy of Federated Learning Model cycle {i*'*'}")

Test Accuracy of Federated Learning Model cycle 
Test Accuracy of Federated Learning Model cycle *
Test Accuracy of Federated Learning Model cycle **
Test Accuracy of Federated Learning Model cycle ***
Test Accuracy of Federated Learning Model cycle ****


In [12]:
i = 0
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
path = f'weighted_average_model_weights_cycle_{i}.pth'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [17]:
i= 0
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # Assuming you have 2 classes
model.load_state_dict(torch.load(f'weighted_average_model_weights_cycle_{i}.pth'))
accuracy = train_model(model, client_datasets['client_3'], 2, model_name = 'client_1', num_epochs=0, device="cuda",test_only=True )
print(f"Test Accuracy of Federated Learning Model cycle {i}: {accuracy:.2f}%")

Test Accuracy of Federated Learning Model cycle 0: 70.47%


Testing the accuracy on unseen data distributions

In [29]:
batch_size = 64
train_size = int(0.7 * len(client_datasets['client_4']))
val_size = int(0.15 * len(client_datasets['client_4']))
test_size = len(client_datasets['client_4']) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(client_datasets['client_4'], [train_size, val_size, test_size])

logging.debug(f"Train size: {train_size}, Val size: {val_size}, Test size: {test_size}")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



i= 0

weights = ['/content/weighted_average_model_weights_cycle_0.pth','/content/client_3_model_weights.pth']
for w in weights:
  print(f"Test Accuracy of Client_3 model on Client_4 data with weights {w}")
  model = models.resnet18(pretrained=True)
  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, 2)  # Assuming you have 2 classes
  model.load_state_dict(torch.load(w))
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)

  num_ftrs = model.fc.in_features
  model.fc = nn.Linear(num_ftrs, 2)
  model.to('cuda')
  model.eval()
  test_correct = 0
  test_total = 0
  for images, labels in test_loader:
      images, labels = images.to('cuda'), labels.to('cuda')
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
      test_total += labels.size(0)
      test_correct += (predicted == labels).sum().item()
  test_accuracy = 100 * test_correct / test_total
  print(f"Test Accuracy with weights {w}: {accuracy:.2f}%")

Test Accuracy of Client_3 model on Client_4 data with weights /content/weighted_average_model_weights_cycle_0.pth
Test Accuracy with weights /content/weighted_average_model_weights_cycle_0.pth: 70.47%
Test Accuracy of Client_3 model on Client_4 data with weights /content/client_3_model_weights.pth
Test Accuracy with weights /content/client_3_model_weights.pth: 70.47%


In [27]:
client_datasets

{'client_1': <__main__.CustomImageDataset at 0x7b1763c76ef0>,
 'client_2': <__main__.CustomImageDataset at 0x7b1763c75cf0>,
 'client_3': <__main__.CustomImageDataset at 0x7b1763c6a080>,
 'client_4': <__main__.CustomImageDataset at 0x7b1763c6a2f0>}